# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_key import g_key

### Store Part I results into DataFrame

In [2]:
# Storing filepath into a variable
file = "../WeatherPy/Output Data/weather_data.csv"

# Reading the filepath and converting it to a dataframe
weather_df = pd.read_csv(file)
weather_df.drop("Unnamed: 0", axis=1, inplace = True)

weather_df

,City,Country,Date,Latitude,Longitude,Temperature,Max. Temp.,Humidity,Cloudiness,Wind Speed
0,Port Alfred,ZA,1600108947,-33.59,26.89,14.68,14.68,67,0,8.78
1,Dunda,TZ,1600108947,-8.12,34.25,22.13,22.13,49,57,1.87
2,Preston,GB,1600108862,53.77,-2.72,20.56,22.00,60,80,2.60
3,Ribeira Grande,PT,1600108948,38.52,-28.70,21.00,21.00,52,20,7.70
4,Mossamedes,AO,1600108948,-15.20,12.15,18.59,18.59,84,100,3.07
...,...,...,...,...,...,...,...,...,...,...
558,Smithers,CA,1600109103,54.78,-127.17,11.00,11.00,76,90,1.00
559,Kirakira,SB,1600109130,-10.45,161.92,26.33,26.33,83,100,7.34
560,Boende,CD,1600109130,-0.22,20.87,22.71,22.71,95,100,0.59
561,Zarubino,RU,1600109131,42.63,131.09,12.75,12.75,84,0,2.99


### Humidity Heatmap

In [3]:
# Configuring gmaps with my google api key
gmaps.configure(api_key = g_key)

In [4]:
# Setting "locations" and "weights" variables to create weighted heatmap
locations = weather_df[["Latitude", "Longitude"]]
weights = weather_df["Humidity"]

# Setting map details
figure_layout = {"width": "100%", "height": "400px", "padding": "1px"}

# Generating heatmap
fig = gmaps.figure(layout=figure_layout, center = (20.0, 5.0), zoom_level = 1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False)
fig.add_layer(heatmap_layer)

# Setting the max humidity value as the max intensity
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria

##### Creating a new df based on the following conditions: 
1. Temperature between 20 and 30 °C
2. Cloudiness below 10%
3. Humidity below 70%
4. Wind speeds below 3 m/s

In [5]:
# Setting conditions to select appropiate cities and dropping null values
best_cities_df = weather_df.loc[(weather_df["Max. Temp."] > 19) &
                                (weather_df["Max. Temp."] < 31) &
                                (weather_df["Cloudiness"] < 10) & 
                                (weather_df["Humidity"] < 70) & 
                                (weather_df["Wind Speed"] < 3)].reset_index(drop = True).dropna()

best_cities_df.head()

,City,Country,Date,Latitude,Longitude,Temperature,Max. Temp.,Humidity,Cloudiness,Wind Speed
0,Hilo,US,1600108952,19.73,-155.09,21.60,26.00,69,1,1.50
1,Albany,US,1600108954,42.60,-73.97,20.76,21.67,40,0,0.45
2,Gazojak,TM,1600108959,41.19,61.40,20.41,20.41,24,0,2.01
3,Bereket,TM,1600108968,39.24,55.52,23.28,23.28,40,0,2.86
4,Mount Isa,AU,1600108968,-20.73,139.50,21.00,21.00,30,0,2.10


### Hotel Map

In [6]:
# Copying info to new dataframe and adding a new blank column
hotel_df = best_cities_df[["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = np.nan

hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Hilo,US,19.73,-155.09,NaN
1,Albany,US,42.60,-73.97,NaN
2,Gazojak,TM,41.19,61.40,NaN
3,Bereket,TM,39.24,55.52,NaN
4,Mount Isa,AU,-20.73,139.50,NaN


In [7]:
# Setting base url 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting parameters
search = "hotel"
target_type = "lodging"
radius = 5000

# Setting parameter dict
parameters = {"keyword": search, 
              "radius": radius, 
              "type": target_type, 
              "key": g_key}

In [8]:
# Look for every city in the dataframe and make the api call

for index, row in hotel_df.iterrows():

    latitude = row["Latitude"]
    longitude = row["Longitude"]
    parameters["location"] = f"{latitude}, {longitude}"
    
    # Make the api call and get hotel details
    hotel_data = requests.get(base_url, parameters).json()
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"Hotel in {row['City']} found | {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"No hotels found in {row['City']}")    
        print("-------------------------------------------------------------------------")

Hotel in Hilo found | Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
-------------------------------------------------------------------------
No hotels found in Albany
-------------------------------------------------------------------------
No hotels found in Gazojak
-------------------------------------------------------------------------
No hotels found in Bereket
-------------------------------------------------------------------------
Hotel in Mount Isa found | ibis Styles Mt Isa Verona
-------------------------------------------------------------------------
Hotel in Margate found | Margate Bay Guest House
-------------------------------------------------------------------------
Hotel in Senanga found | Zambia
-------------------------------------------------------------------------
Hotel in Cham found | City Garden Hotel
-------------------------------------------------------------------------
No hotels found in Abu Samrah
----------------------------------------------------

In [9]:
# Dropping rows where no hotel was found and printing results
hotel_df.dropna(inplace = True)
hotel_df.reset_index(drop = True, inplace = True)

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Hilo,US,19.73,-155.09,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
1,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
2,Margate,GB,51.38,1.39,Margate Bay Guest House
3,Senanga,ZM,-16.12,23.27,Zambia
4,Cham,CH,47.18,8.46,City Garden Hotel
5,Dalbandin,PK,28.89,64.41,Al-Dawood Hotel & Restaurant
6,Arrondissement de Saint-Denis,FR,48.92,2.33,ART hotel paris est
7,Tucumcari,US,35.17,-103.72,Fairfield Inn & Suites by Marriott Tucumcari
8,Yangi Marg`ilon,UZ,40.43,71.72,Hotel Mehmon Saroy
9,Wajir,KE,1.75,40.06,The Aaran Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))